In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential
import plotly.express as px

# Define the date range
start_date = '2008-01-01'
end_date = '2022-12-31'

# Fetch TSLA stock data using yfinance
df = yf.download('TSLA', start=start_date, end=end_date)

# Calculate 100-day and 200-day moving averages
ma100 = df['Close'].rolling(100).mean()
ma200 = df['Close'].rolling(200).mean()

# Plot stock data with moving averages
fig = px.line(df, x=df.index, y='Close', title='TSLA Stock Price with Moving Averages')
fig.add_scatter(x=df.index, y=ma100, mode='lines', name='100-day MA', line=dict(color='red'))
fig.add_scatter(x=df.index, y=ma200, mode='lines', name='200-day MA', line=dict(color='green'))
fig.show()

# Drop unnecessary columns
df = df.drop(['Open', 'High', 'Low', 'Volume'], axis=1)

# Split the data into training and testing sets
training_data = pd.DataFrame(df['Close'][0:int(len(df) * 0.70)])
testing_data = pd.DataFrame(df['Close'][int(len(df) * 0.70):])

print(f"Training Data Shape: {training_data.shape}")
print(f"Testing Data Shape: {testing_data.shape}")

# Scale the training data
scaler = MinMaxScaler(feature_range=(0, 1))
training_data_array = scaler.fit_transform(training_data)

# Create sequences for training
x_train = []
y_train = []

for i in range(100, training_data_array.shape[0]):
    x_train.append(training_data_array[i - 100: i])
    y_train.append(training_data_array[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=60, activation='relu', return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(units=80, activation='relu', return_sequences=True))
model.add(Dropout(0.4))

model.add(LSTM(units=120, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=1))

# Summary of the model
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, epochs=50)

# Save the trained model
model.save('keras_model.h5')


[*********************100%%**********************]  1 of 1 completed


Training Data Shape: (2205, 1)
Testing Data Shape: (945, 1)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 50)           10400     
                                                                 
 dropout (Dropout)           (None, 100, 50)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100, 60)           26640     
                                                                 
 dropout_1 (Dropout)         (None, 100, 60)           0         
                                                                 
 lstm_2 (LSTM)               (None, 100, 80)           45120     
                                                                 
 dropout_2 (Dropout)         (None, 100, 80)           0         
                                                              

In [3]:
import plotly.graph_objects as go

In [4]:
# Prepare testing data
testing_data = pd.DataFrame(df['Close'][int(len(df) * 0.70) - 100:])
input_data = scaler.fit_transform(testing_data)

x_test = []
y_test = []

for i in range(100, input_data.shape[0]):
    x_test.append(input_data[i - 100: i])
    y_test.append(input_data[i, 0])

x_test, y_test = np.array(x_test), np.array(y_test)
print(f"Testing Data Shapes: {x_test.shape}, {y_test.shape}")

# Make predictions
y_predicted = model.predict(x_test)

# Inverse scaling to get original price values
scaler_factor = 1 / 0.00251232
y_predicted = y_predicted * scaler_factor
y_test = y_test * scaler_factor

# Plotting results using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index[int(len(df) * 0.70):], y=y_test, mode='lines', name='Original Price'))
fig.add_trace(go.Scatter(x=df.index[int(len(df) * 0.70):], y=y_predicted.flatten(), mode='lines', name='Predicted Price'))
fig.update_layout(title='TSLA Stock Price Prediction')
fig.update_xaxes(title='Time')
fig.update_yaxes(title='Price')
fig.show()

Testing Data Shapes: (945, 100, 1), (945,)
30/30 [==============================] - 3s 95ms/step
